# Calculating composite MCWD and MAP values

In Duque et al. (2019), we have created a new kind of plot in which we display together the maximum climatological water deficit (MCWD) and mean annual precipitation (MAP) differences between two experiment simulations. We believe this kind of plot shows us valuable information about what land world regions are more vulnerable to experiment scenarios. In our experimental module we have a function to help us create this plot. Let's import obrero and the module:

In [1]:
# small hack to be able to import module without install
import os
import sys
sys.path.append(os.getcwd() + '/../')

import obrero
from obrero.experimental import mcwd

Now we need data from two simulations. As well as an evaporation file. We always use the evaporation values from the control simulation because we think those values should not change if land cover stays the same. That is to say that energy demand from what is an ecosystem should not change if the ecosystem is to be preserved:

In [3]:
# file name
f1 = 'data/ctl_pr_evap.nc'
f2 = 'data/pen_pr_evap.nc'
f3 = 'data/ctl_evapmean.nc'

# read as data array (ignore warnings because netCDF files are weird)
da1 = obrero.read_nc(f1, 'pr')
da2 = obrero.read_nc(f2, 'pr')
ev = obrero.read_nc(f3, 'evap')

First we have to get MAP, which is the sum of all precipitation in every year:

In [4]:
map1 = da1.groupby('time.year').sum(dim='time', keep_attrs=True)
map2 = da2.groupby('time.year').sum(dim='time', keep_attrs=True)

# rename time
map1 = map1.rename({'year': 'time'})
map2 = map2.rename({'year': 'time'})

Next we calculate MCWD for each experiment using the same evaporation in both of them:

In [5]:
# compute mcwd
wd1 = mcwd.get_mcwd(da1, ev)
wd2 = mcwd.get_mcwd(da2, ev)

To get the composite map values we use function `mcwd_composite_map()` in the experimental `obrero.experimental.mcwd` module. The order of the arrays matters: first MCWD and MAP from an experiment and then the same for the **control** experiment. This is because there will be substractions. It will return a gridded map as well as a data frame with all values tabulated:

In [9]:
composite, table = mcwd.mcwd_composite_map(wd2, map2, wd1, map1)

Let's look at these two things:

In [10]:
composite

<xarray.DataArray 'composite' (latitude: 32, longitude: 64)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * latitude   (latitude) float64 85.76 80.27 74.74 ... -74.74 -80.27 -85.76
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
Attributes:
    long_name:  Composite MAP and MCWD
    units:      1

This is the map which is used for plotting purposes. And then the table:

In [11]:
table

,lon,lat,ctl_map,exp_map,ctl_mcwd,exp_mcwd,comp
id,,,,,,,
0,236.250,69.212976,568.175017,628.333967,-89.667515,-67.413326,5.5
1,241.875,69.212976,584.077378,644.068658,-76.084100,-59.616325,5.5
2,39.375,58.142954,791.212306,886.235404,-68.037802,-29.971604,5.5
3,22.500,52.606526,790.350410,891.052641,-184.560142,-119.409784,5.5
4,33.750,52.606526,419.365649,484.679045,-158.039537,-134.594018,5.5
5,39.375,52.606526,522.831663,592.652493,-181.348686,-149.360635,5.5
6,50.625,52.606526,585.916357,666.182773,-122.353594,-102.642326,5.5
7,95.625,52.606526,1087.879558,1216.880009,-12.081088,-1.436797,5.5
8,5.625,47.069642,684.800756,807.632279,-135.093102,-71.060846,5.5


The composite value can be one of 0.5, 1.5, 2.5, 3.5, 4.5, or 5.5. The meaning of each of these values is explained in detail in Duque et al. (2019).

## References

Duque et al. (2019).